# میام تعداد داده ای که لازم داریم برای آنالیز تکنیکالی به لیمیت میدیم و تاریخ و ساعت الان هم میگیرم و سپس میام اون تعداد لیمیت براساس زمان از تاریخ و ساعتی که براساس میکروثانیه هست کم میکنیم و از اونجا زمان و تاریخ شروع بدست آوردن داده ها را بدست میاریم 

In [ ]:
import ccxt
import calendar
from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
binance = ccxt.binance()

def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple())*1000
    ohlcv1 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    #ohlcv2 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv = ohlcv1 #+ ohlcv2
    return ohlcv
m=121
def ohlcv(dt, pair, period):
    ohlcv = []
    limit = 1000
    if period == '1m':
        limit = 750
    elif period == '1d':
        limit = 400
    elif period == '1h':
        limit = 24
    elif period == '5m':
        limit = 288
    for i in dt:
        start_dt = datetime.strptime(i, "%Y%m%d")
        since = calendar.timegm(start_dt.utctimetuple())*1000
        #if period == '1m':
            #ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        #else:
        ohlcv.extend(binance.fetch_ohlcv(symbol=pair, timeframe=period, since=since, limit=limit))
    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)
    df.set_index('Time', inplace=True)
    return df

dt = ['20180101']
df = ohlcv(dt, 'PEOPLE/USDT', '1d')
pd.set_option('display.max_rows', df.shape[0]+1)
df

In [ ]:
#این برای گرفتن داده ها برای یک مدت زمان خاصی است
import ccxt
import calendar
from datetime import datetime

binance = ccxt.binance()

now = datetime.utcnow()

unixtime = calendar.timegm(now.utctimetuple())
since = (unixtime - 60*60) * 1000 # UTC timestamp in milliseconds (60*60=1h)
print(now,"**",unixtime,"**",since)
ohlcv = binance.fetch_ohlcv(symbol='BTC/USDT', timeframe='5m', since=since, limit=12)
start_dt = datetime.fromtimestamp(ohlcv[0][0]/1000)
end_dt = datetime.fromtimestamp(ohlcv[-1][0]/1000)

# convert it into Pandas DataFrame
import pandas as pd

df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
df.set_index('Time', inplace=True)
df